In [1]:
import string
import re
from itertools import product
import numpy as np

In [37]:
#Read the English_Updated.txt file
fh=open('English_Updated.txt',encoding="utf8")
eng = []
for j, line in enumerate(fh):
    s = line.lstrip().rstrip()
    if s != '':
        eng.append(s)
        j += 1
fh.close()

translator = str.maketrans('', '', '''!"#$%&\()*+,-./:;<=>?@[\\]^_`{|}~0123456789''')        

finalEng = []
for line in eng:
    out=line.translate(translator).replace("' s", "").replace("' ","'")
    finalEng.append(out.lower())

#Read the Dutch_Updated.txt file
fh1=open('Dutch_Updated.txt',encoding="utf8")
dutch = []

for j, line in enumerate(fh1):
    s = line.lstrip().rstrip()
    if s != '':
        dutch.append(s)
        j += 1
fh1.close()

finalDutch = []
for line in dutch:
    out = line.translate(translator)
    finalDutch.append(out.lower())

#ps is the list which contains the parallel sentences of English and Dutch corpus
ps = []
for i in range(len(finalEng)):
    ps.append((finalEng[i],finalDutch[i]))

#Remove the unwanted variables and data structures from the memory
del eng 
del finalEng
del dutch
del finalDutch


In [41]:
#convert the list ps into numpy array
pair_sent = np.array(ps)

In [44]:
prev1 = 0
prev2 = 0

#dictionaryMatEng: It is the dictionary which maps an English word to a number
dictionaryMatEng = {}

#dictionaryMatDut: It is the dictionary which maps a Dutch word to a number
dictionaryMatDut = {}

#invDictionaryMatEng: It is the dictionary which maps a number to an English word
invDictionaryMatEng = {}

#invDictionaryMatDut: It is the dictionary which maps a number to a Dutch word
invDictionaryMatDut = {}

#Now, create the dictionary and inverse dictionaries
for i in range(len(pair_sent)):
    A = pair_sent[i][0].split()
    B = pair_sent[i][1].split()
    
    for word in A:
        if word not in dictionaryMatEng:
            dictionaryMatEng[word] = prev1
            invDictionaryMatEng[prev1] = word
            prev1 += 1


    for word in B:
        if word not in dictionaryMatDut:
            dictionaryMatDut[word] = prev2
            invDictionaryMatDut[prev2] = word
            prev2 += 1

#mat_l: Number of rows in the matrix, mat
#mat_b: Number of columns in the matrix, mat
mat_l, mat_b = prev1, prev2


#Delete the unused variables
del prev1
del prev2

In [45]:
mat_l, mat_b

(37302, 60544)

In [46]:
#mat: This is the matrix which will store the probability
mat = np.array([[1/mat_l]*mat_b for y in range(mat_l)])

In [47]:
#EM Algorithm

#We run the algorithm for 50 iterations
for num in range(50):
    total = np.zeros(mat_b)
    
    count = np.zeros((mat_l, mat_b))
        
    #Expectation part
    
    
    for i in range(len(pair_sent)):
        en = pair_sent[i][0].split()
        dut = pair_sent[i][1].split()
        s_total = np.zeros(len(en))
        
        #Here alignment generation is done(implicit, not stored anywhere due to memory constraints)
        for i in range(len(en)):
            for j in range(len(dut)):    
                s_total[i] += mat[ dictionaryMatEng[en[i]] ][ dictionaryMatDut[dut[j]] ]
             
            for j in range(len(dut)):
                value = mat[ dictionaryMatEng[en[i]] ][ dictionaryMatDut[dut[j]] ]
                count [ dictionaryMatEng[en[i]] ][ dictionaryMatDut[dut[j]] ] += value/s_total[i]
                total [ dictionaryMatDut[dut[j]] ] += value/s_total[i]

                
    #Maximization
    #Now, we'll update the mat matrix
    mat = np.where(total!=0, count/total, 0)

    if num%5==0:
        print(num)

0
5
10
15
20
25
30
35
40
45


In [ ]:
#IBM Model2

#Run the IBM Model 2 
for num in range(50):
    total = np.zeros(mat_b)
    
    count = np.zeros((mat_l, mat_b))
        
    q = np.zeros((50, 50, 50, 50))
    c1 = np.zeros((50, 50, 50, 50))
    c2 = np.zeros((50, 50, 50))
    
    #Expectation
    for i in range(len(pair_sent)):
        en = pair_sent[i][0].split()
        dut = pair_sent[i][1].split()
        s_total = np.zeros(len(en))
        
        
        for i in range(len(en)):
            for j in range(len(dut)):
                s_total[i] += (mat[ dictionaryMatEng[en[i]] ][ dictionaryMatDut[dut[j]] ] * q[i][j][len(en)][len(dut)])
             
            for j in range(len(dut)):
                value = (mat[ dictionaryMatEng[en[i]] ][ dictionaryMatDut[dut[j]] ] * q[i][j][len(en)][len(dut)])
                count [ dictionaryMatEng[en[i]] ][ dictionaryMatDut[dut[j]] ] += value/s_total[i]
                total [ dictionaryMatDut[dut[j]] ] += value/s_total[i]
                c1[i][j][len(en)][len(dut)] += value/s_total[i]
                c2[j][len(en)][len(dut)] += value/s_total[i]
                

    #Maximization
    q = np.where(c2!=0, c1/c2, 0)
    if num%5==0:
        print(num)

In [48]:
#finalDictionary: It is the dictionary which stores the acutal dictionary generated from the model
finalDictionary = {}
for i in range(mat_l):
    index_max = np.argmax(mat[i])
    finalDictionary[ invDictionaryMatEng[i] ] = invDictionaryMatDut[index_max]

In [49]:
print(finalDictionary)

{'as': 'zoals', 'i': 'ik', 'said': 'zei', 'the': 'the', 'matters': 'huwelijkszaken', 'that': 'dat', 'we': 'wij', 'are': 'bevinden', 'discussing': 'energieconsumerende', 'today': 'vandaag', 'and': 'en', 'subject': 'onderworpen', 'of': 'van', 'central': 'centrale', 'bank': 'bank', 'report': 'verslag', 'affect': 'bouwbedrijven', 'daily': 'dagelijkse', 'life': 'life', 'millions': 'miljoenen', 'people': 'mensen', 'mr': 'heer', 'president': 'president', 'come': 'basics', 'from': 'vanaf', 'a': 'een', 'very': 'zeer', 'small': 'kleine', 'village': 'dorp', 'with': 'geklets', 'total': 'total', 'inhabitants': 'inwoners', 'in': 'in', 'smallest': 'kleinste', 'member': 'afgevaardigde', 'state': 'staatssteun', 'european': 'europees', 'union': 'unie', 'namely': 'bankwetgeving', 'malta': 'malta', 'nevertheless': 'niettemin', 'what': 'wat', 'debating': 'debatteert', 'such': 'dergelijke', 'pompous': '“verdrag', 'words': 'woorden', 'sometimes': 'soms', 'affects': 'welbehagen', 'lives': 'levens', 'families'

In [50]:
#Convert the finalDictionary to the pandas.DataFrame
import pandas as pd

Word_Dict = pd.DataFrame(list(finalDictionary.items()), columns=['Eng_Word', 'Dutch_Word'])
Word_Dict[:10]

,Eng_Word,Dutch_Word
0,as,zoals
1,i,ik
2,said,zei
3,the,the
4,matters,huwelijkszaken
5,that,dat
6,we,wij
7,are,bevinden
8,discussing,energieconsumerende
9,today,vandaag


In [51]:
len(Word_Dict)

37302

In [52]:
#Save the dictionary as the .csv file
Word_Dict.to_csv('EM_Dictionary5.csv')

In [ ]:
wordict = {}
tempdict = {}

for word in tempdict.keys():
    wordict[word] = tempdict[word]